In [114]:
from matplotlib import pyplot as plt 
from PIL import Image, ImageDraw
from ultralytics import YOLO
from tqdm import tqdm
import numpy as np
import IPython
import cv2
import os

model = YOLO('runs/segment/train_export_large/weights/best.pt')  # load a pretrained model (recommended for training)

save_result= True

source_dir = "Patrick_images/images"


def delete_all_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                delete_all_files_in_folder(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))


counter = 0
alpha_value = 100
outline_alpha_value = 150

color_list =    [
                (102, 51, 0, alpha_value), # TVF
                (102, 0, 102, alpha_value), # Teto
                (0, 255, 0, alpha_value),# Escadas
                (0, 0, 255, alpha_value),# Estruturas
                (250, 0, 0, alpha_value),# Equipamanto
                (255, 153, 0, alpha_value), # Guardacorpo
                (255, 102, 204, alpha_value), # Piso
                (0, 51, 0, alpha_value), # Suportes
                (0, 0, 0, alpha_value),
            ] 

outline_list =    [
                (102, 51, 0, outline_alpha_value), # TVF
                (102, 0, 102, outline_alpha_value), # Teto
                (0, 255, 0, outline_alpha_value), # Escadas
                (0, 0, 255, outline_alpha_value), # Estruturas
                (250, 0, 0, outline_alpha_value), # Equipamanto
                (255, 153, 0, outline_alpha_value), # Guardacorpo
                (255, 102, 204, outline_alpha_value), # Piso
                (0, 51, 0, outline_alpha_value), # Suportes
                (0, 0, 0, outline_alpha_value), 
            ] 

delete_all_files_in_folder('Patrick_images/inference_output')

not_object_list = []
frame_list = []

for file_name in tqdm(os.listdir(source_dir)):
    img_name = file_name
    results = model(f'Patrick_images/images/{img_name}', save = False)  # predict on an image

    # Load your RGB image
    image = Image.open(f'Patrick_images/images/{img_name}')
    width,height = image.size

    image = image.convert("RGBA")

    # Create a new image with an alpha channel
    image_with_alpha = Image.new("RGBA", image.size)


    complete_image = np.zeros((height,width,4))

    # Create a drawing context for the image with an alpha channel
    draw = ImageDraw.Draw(image_with_alpha)

    for result in results:
        if result is not None:
            if result.masks is not None:
                masks = result.masks.xy  # Masks object for segmentation masks outputs
                classes = result.boxes.cls  # Masks object for segmentation masks outputs
                for i in range(len(masks)):
                    mask = masks[i]
                    
                    classe = int(classes[i].item())
                    fill_color = color_list[classe]
                    outline_color = outline_list[classe]
                    polygon_point_list = []
                    for point in mask:
                        x,y = point
                        polygon_point_list.append((x,y))
                        
                    if len(polygon_point_list) > 1:
                        if classe <100:
                            #print(f"classe = {classe}")
                            #print(f"polygon_point_list = {polygon_point_list}")
                            draw.polygon(polygon_point_list, fill=fill_color, outline=(outline_color),width=5)
                    else:
                        #(mask)
                        continue
                    
            else:
                not_object_list.append(f'Patrick_images/images/{img_name}')
        annotated_face = Image.alpha_composite(image, image_with_alpha)
        img = np.array(annotated_face) 
        img[..., :3] = img[..., 2::-1]

    if save_result: 
        #remove the ".png" from the filename
        img_name = img_name[:-4]
        
        if img_name.split("_")[-1]=="0":
            frame_list.append(img_name[:-1])
            img_name = img_name[:-1]+"negx"

        elif img_name.split("_")[-1]=="90":
            frame_list.append(img_name[:-3])
            img_name = img_name[:-2]+"posz"

        elif img_name.split("_")[-1]=="180":
            frame_list.append(img_name[:-3])
            img_name = img_name[:-3]+"posx"

        elif img_name.split("_")[-1]=="270":
            frame_list.append(img_name[:-3])
            empty_face = np.zeros_like(img) 
            empty_face_mask = np.zeros_like(np.array(image_with_alpha) ) 
            img_name = img_name[:-3]
            #cv2.imwrite(f'Patrick_images/inference_output/cubemap_{img_name}posy.png',empty_face)
            #cv2.imwrite(f'Patrick_images/inference_output/cubemap_{img_name}negy.png',empty_face)
            #cv2.imwrite(f'Patrick_images/inference_output/cubemap_mask_{img_name}posy.png',empty_face)
            #cv2.imwrite(f'Patrick_images/inference_output/cubemap_mask_{img_name}negy.png',empty_face)
            img_name = img_name+"negz"
            
        cv2.imwrite(f'Patrick_images/inference_output/cubemap_{img_name}.png',img)
        annotations_only = np.array(image_with_alpha) 
        cv2.imwrite(f'Patrick_images/inference_output/cubemap_mask_{img_name}.png',annotations_only)
        
        
    
    
print(not_object_list)



  0%|          | 0/593 [00:00<?, ?it/s]
image 1/1 f:\Pedro\YOLOv8\Darwin_dados\Patrick_images\images\000001_image_1024_0_0.png: 640x640 4 TVFs, 1 Estruturas, 1 Equipamento, 3 Pisos, 5 Suportess, 38.5ms
Speed: 7.0ms preprocess, 38.5ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
  0%|          | 1/593 [00:01<18:12,  1.85s/it]
image 1/1 f:\Pedro\YOLOv8\Darwin_dados\Patrick_images\images\000001_image_1024_0_180.png: 640x640 9 TVFs, 1 Estruturas, 1 Equipamento, 2 Pisos, 4 Suportess, 37.0ms
Speed: 6.0ms preprocess, 37.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
  0%|          | 2/593 [00:02<09:17,  1.06it/s]
image 1/1 f:\Pedro\YOLOv8\Darwin_dados\Patrick_images\images\000001_image_1024_0_270.png: 640x640 3 TVFs, 9 Escadass, 1 Equipamento, 11 Guarda Corpos, 1 Piso, 2 Suportess, 23.7ms
Speed: 19.7ms preprocess, 23.7ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)
  1%|          | 3/593 [00:02<06:48,  1.45it/s]
image 1/1 f:\Pedro

['Patrick_images/images/002031_image_1024_0_270.png']


In [133]:
def make_video_from_folder(folder_name, face_name):
    img = cv2.imread(f"{folder_name}/cubemap_000001_image_1024_0_negx.png")
    height,width,layers = img.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    # Check fps if necessary
    video = cv2.VideoWriter(f'video_{face_name}.mp4', fourcc, 3, (width, height))

    frame_count=0
    for i in range(10000,0,-1):
        try:
            img = cv2.imread(f"{folder_name}/cubemap_{i:06}_image_1024_0_{face_name}.png")
            if img is not None:
                frame_count +=1
                video.write(img)
        except:
            pass    
    print(frame_count)
    #cv2.destroyAllWindows()
    video.release()

for face in ['negx', 'negz', 'posx', 'posz']:
    make_video_from_folder("Patrick_images/inference_output",face)

149
149
149
146
